In [56]:
import pandas as pd
from re import sub

In [57]:
def preprocess(text):
    text = text.lower().strip()
    text = sub('[áàãâä]', 'a', sub('[éèêë]', 'e', sub('[íìîï]', 'i', sub('[óòõôö]', 'o', sub('[úùûü]', 'u', sub('[ç]','c',text))))))
    text = sub(r'\s+', ' ',text)
    return text

In [58]:
df_excel_pt = pd.read_excel('troubleshooting_equipament.xlsx')
df_excel_en = pd.read_excel('troubleshooting_equipament_en.xlsx')

In [59]:
df_pt = df_excel_pt.astype(str).apply(lambda x: x.apply(preprocess))
df_en = df_excel_en.astype(str).apply(lambda x: x.apply(preprocess))

In [60]:
# df_pt.head()
df_en.head()

,id,pag,type,equipment,problem,cause,action,
0,0,1,mechanical,pumps,pump too hot and flashing off,unloading valve not working properly,repair or replace the unloading valve,
1,1,1,mechanical,pumps,pump too hot and flashing off,excessive wear in pump,repair or replace the pump,
2,2,1,mechanical,pumps,pump too hot and flashing off,relief valve set too high,adjust the relief valve,
3,3,1,mechanical,pumps,pump too hot and flashing off,discharge valve throttled or closed,open the discharge valve,
4,4,1,mechanical,pumps,pump churning up fluid and failing to discharg...,leak in suction pipe,inspect the line for broken or loose connectio...,


### problem

In [61]:
df_problems_pt = pd.DataFrame(df_pt["problem"].unique(), columns=["issue_description_pt"])
df_problems_en = pd.DataFrame(df_en["problem"].unique(), columns=["issue_description_en"])
df_problems = pd.concat([df_problems_pt, df_problems_en], axis=1)
df_problems['problem_id'] = df_problems.index
display(df_problems.head())
df_problems.shape

,issue_description_pt,issue_description_en,problem_id
0,bomba muito quente e piscando,pump too hot and flashing off,0
1,bomba agitando o fluido e nao descarregando a ...,pump churning up fluid and failing to discharg...,1
2,bomba que agita o fluido e nao descarrega a ag...,pump churning up fluid and failing to discharg...,2
3,fortes vibracoes durante a operacao da bomba,heavy vibrations during the operation of the pump,3
4,vibracoes fortes durante a operacao da bomba (...,heavy vibrations during the operation of the p...,4


(153, 3)

### Cause

In [104]:
df_causes = pd.DataFrame(df_pt["cause"].unique(), columns=["cause_pt"])
df_causes['cause_id'] = df_causes.index
display(df_causes.head())
df_causes.shape

,cause_pt,cause_id
0,a valvula de descarga nao esta funcionando cor...,0
1,desgaste excessivo na bomba,1
2,valvula de alivio ajustada muito alta,2
3,valvula de descarga estrangulada ou fechada,3
4,vazamento no tubo de succao,4


(483, 2)

In [70]:
df_causes.to_excel('aaa.xlsx')

### action

In [63]:
df_action_pt = pd.DataFrame(df_pt["action"].unique(), columns=["resolution_action_pt"])
df_action_en = pd.DataFrame(df_en["action"].unique(), columns=["resolution_action_en"])
df_action = pd.concat([df_action_pt, df_action_en], axis=1)
df_action['action_id'] = df_action.index
display(df_action.head())
df_action.shape

,resolution_action_pt,resolution_action_en,action_id
0,reparar ou substituir a valvula de descarga,repair or replace the unloading valve,0
1,reparar ou substituir a bomba,repair or replace the pump,1
2,ajuste a valvula de alivio,adjust the relief valve,2
3,abrir a valvula de descarga,open the discharge valve,3
4,inspecionar a linha quanto a conexoes quebrada...,inspect the line for broken or loose connectio...,4


(449, 3)

### equipment

In [64]:
df_equipment_pt = pd.DataFrame(df_pt["equipment"].unique(), columns=["equipment_name_pt"])
df_equipment_en = pd.DataFrame(df_en["equipment"].unique(), columns=["equipment_name_en"])
df_equipment = pd.concat([df_equipment_pt, df_equipment_en], axis=1)
df_equipment['equipment_id'] = df_equipment.index
display(df_equipment.head())
df_equipment.shape

,equipment_name_pt,equipment_name_en,equipment_id
0,bombas,pumps,0
1,rolamentos,bearings,1
2,valvulas,valves,2
3,acionamentos por corrente,chain drives,3
4,caixas de engrenagens,gear boxes,4


(15, 3)

### troubleshooting

In [105]:
# Criação do DataFrame para a tabela Troubleshooting
df_troubleshooting = df_pt[['type', 'equipment', 'problem', 'cause', 'action']].copy()
df_troubleshooting['equipment_id'] = df_troubleshooting['equipment'].map(df_equipment.set_index('equipment_name_pt')['equipment_id'])
df_troubleshooting['problem_id'] = df_troubleshooting['problem'].map(df_problems.set_index('issue_description_pt')['problem_id'])
df_troubleshooting['cause_id'] = df_troubleshooting['cause'].map(df_causes.set_index('cause_pt')['cause_id'])
df_troubleshooting['action_id'] = df_troubleshooting['action'].map(df_action.set_index('resolution_action_pt')['action_id'])
df_troubleshooting = df_troubleshooting[['type', 'equipment_id', 'problem_id', 'cause_id', 'action_id']]

df_troubleshooting['equipment_id'] = df_troubleshooting['equipment_id'].astype(int)
df_troubleshooting['problem_id'] = df_troubleshooting['problem_id'].astype(int)
df_troubleshooting['cause_id'] = df_troubleshooting['cause_id'].astype(int)
df_troubleshooting['action_id'] = df_troubleshooting['action_id'].astype(int)

df_troubleshooting[['equipment_id', 'problem_id', 'cause_id', 'action_id']] = df_troubleshooting[['equipment_id', 'problem_id', 'cause_id', 'action_id']].applymap(lambda x: int(x) + 1)

display(df_troubleshooting.head())
df_troubleshooting.shape

,type,equipment_id,problem_id,cause_id,action_id
0,mecanico,1,1,1,1
1,mecanico,1,1,2,2
2,mecanico,1,1,3,3
3,mecanico,1,1,4,4
4,mecanico,1,2,5,5


(536, 5)

### banco

In [106]:
# Remover a coluna 'id' dos DataFrames
df_problems = df_problems.drop('problem_id', axis=1)
df_causes = df_causes.drop('cause_id', axis=1)
df_action = df_action.drop('action_id', axis=1)
df_equipment = df_equipment.drop('equipment_id', axis=1)

In [ ]:
df_problems

### Mysql

In [ ]:
import pymysql
from sqlalchemy import create_engine

# Configurações de conexão com o banco de dados MySQL
host = 'localhost'
port = 3306
user = 'root'
password = ''
database = 'equipment_failures'

# Cria a conexão com o banco de dados
conn = pymysql.connect(
    host=host,
    port=port,
    user=user,
    password=password,
    database=database
)

engine = create_engine(f'mysql+pymysql://{user}:{password}@{host}:{port}/{database}')

df_problems.to_sql('problem', engine, if_exists='append', index=False)
df_causes.to_sql('cause', engine, if_exists='append', index=False)
df_action.to_sql('action', engine, if_exists='append', index=False)
df_equipment.to_sql('equipment', engine, if_exists='append', index=False)

df_troubleshooting.to_sql('troubleshooting', engine, if_exists='append', index=False)

conn.close()


### Sqlite

In [109]:
import sqlite3

df_troubleshooting.columns = ['type','equipment_id', 'issue_id', 'cause_id', 'action_id']

# Configurações de conexão com o banco de dados SQLite
database_file = r'bd_test\IndustrialTroubleshootDB.db'

# Cria a conexão com o banco de dados
conn = sqlite3.connect(database_file)

# DataFrames para tabela problem, cause, action, equipment, troubleshooting
df_problems.to_sql('issue', conn, if_exists='append', index=False)
df_causes.to_sql('root_cause', conn, if_exists='append', index=False)
df_action.to_sql('resolution', conn, if_exists='append', index=False)
df_equipment.to_sql('equipment', conn, if_exists='append', index=False)
df_troubleshooting.to_sql('troubleshooting', conn, if_exists='append', index=False)

# Fecha a conexão com o banco de dados
conn.close()